In [1]:
# importing important libraries
import numpy as np
import pandas as pd
import time
from scipy.linalg import lu_factor, lu_solve

In [2]:
# defining lu decomposition function which takes n*n matrix A as 
# input and finds permutation matrix P, lower triangular matrix L, 
# and upper triangular matrix L. 
def LUDecomp(A,n):
    P = np.eye(n)
    L = np.eye(n)
    U = A.copy()
    for col in range(n-1):
        argmax = col
        maxnum = abs(U[col][col])
        for row in range(col,n):
            if abs(U[row][col]) > maxnum:
                maxnum = abs(U[row][col])
                argmax = row
        if maxnum == 0:
            raise ValueError("Matrix is singular")
        Q = np.eye(n)
        Q[col,col] = 0
        Q[argmax,col] = 1
        Q[argmax,argmax] = 0
        Q[col,argmax] = 1
        P = Q @ P
        if argmax != col:
            U[[col, argmax]] = U[[argmax, col]]
            L[[col, argmax], :col] = L[[argmax, col], :col]
        for row in range(col+1,n):
            L[row,col] = U[row,col]/U[col,col]
            for col2 in range(col,n):
                U[row,col2] -= L[row,col] * U[col,col2]
    if U[n-1,n-1] == 0:
        raise ValueError("Matrix is singular")
    return P,L,U

In [3]:
# This function performs forward substitution 
# to solve the lower triangular system Ly=b 
# by computing each element of y sequentially 
# using previously calculated values.
def forwardSubstitution(L,b,n):
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i]
        for j in range(i):
            y[i] -= L[i,j] * y[j]
    return y

In [4]:
# This function performs backward substitution 
# to solve the upper triangular system  Ux=y 
# by computing each element of  from bottom to top 
# using previously computed values
def backwardSubstitution(U,y,n):
    x = np.zeros(n)
    for i in range(n-1,-1,-1):
        x[i] = y[i]
        for j in range(i+1,n):
            x[i] -= U[i,j]*x[j]
        x[i] = x[i] / U[i,i]
    return x

In [5]:
# This function solves the linear system  Ax=b using LU decomposition 
# with partial pivoting by first applying the permutation  P to  b, 
# then solving  Ly=Pb via forward substitution and Ux=y via backward substitution.
def LUSolver(P,L,U,b,n):
    b_permuted = P @ b
    y = forwardSubstitution(L,b_permuted,n)
    x = backwardSubstitution(U,y,n)
    return x

In [6]:
sizes = [100,300,500,1000,2000]

results_factor = []
results_solve = []

for n in sizes:

    # Generate random matrices
    # of size n
    A = np.random.randn(n, n)
    b = np.random.randn(n)

    # -------------------------
    # Factorization timing (My code)
    # -------------------------
    start = time.perf_counter()
    P, L, U = LUDecomp(A, n)
    t_factor_my = time.perf_counter() - start

    # -------------------------
    # LU Solver timing (My Code)
    # -------------------------
    start = time.perf_counter()
    x_my = LUSolver(P, L, U, b, n)
    t_solve_my = time.perf_counter() - start

    # -------------------------
    # SciPy factorization timing
    # -------------------------
    start = time.perf_counter()
    lu, piv = lu_factor(A)
    t_factor_scipy = time.perf_counter() - start

    # -------------------------
    # SciPy LU solver timing
    # -------------------------
    start = time.perf_counter()
    x_scipy = lu_solve((lu, piv), b)
    t_solve_scipy = time.perf_counter() - start

    # -------------------------
    # Accuracy measures
    # -------------------------
    back_error_my = np.linalg.norm(P @ A - L @ U) / np.linalg.norm(A)
    residual_my = np.linalg.norm(A @ x_my - b) / np.linalg.norm(b)

    residual_scipy = np.linalg.norm(A @ x_scipy - b) / np.linalg.norm(b)

    # Storing results
    results_factor.append([n, t_factor_my, t_factor_scipy, back_error_my])
    results_solve.append([n, t_solve_my, t_solve_scipy, residual_my, residual_scipy])


In [11]:
df_factor = pd.DataFrame(results_factor,
    columns=["n",
             "Time (My LU)",
             "Time (SciPy LU)",
             "Backward Error (My Code)"])

df_solve = pd.DataFrame(results_solve,
    columns=["n",
             "Time (My Solver)",
             "Time (SciPy Solver)",
             "Residual (My Code)",
             "Residual (SciPy)"])

print(df_factor.to_string())
print(df_solve.to_string())

      n  Time (My LU)  Time (SciPy LU)  Backward Error (My Code)
0   100      0.412878         0.002897              7.691991e-16
1   300      8.859576         0.002378              1.892832e-15
2   500     42.344367         0.002996              2.869635e-15
3  1000    299.100981         0.058121              5.265080e-15
4  2000   2258.511319         0.168677              9.445059e-15
      n  Time (My Solver)  Time (SciPy Solver)  Residual (My Code)  Residual (SciPy)
0   100          0.009154             0.000267        9.622246e-15      8.073787e-15
1   300          0.075898             0.000078        5.340335e-14      6.338166e-14
2   500          0.216014             0.000184        5.485706e-13      4.513142e-13
3  1000          0.609062             0.000549        6.013045e-13      6.506382e-13
4  2000          2.390152             0.002384        2.468988e-12      2.494769e-12
